<a href="https://colab.research.google.com/github/gustavoalcantara/portfolio_data/blob/main/idhm_presidentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IN PROGRESS
pip install basedosdados -q

In [4]:
import basedosdados as bd

In [9]:
#Lendo/Acessando o df. a partir do SQL e pacote da Base dos Dados para 2018 no 2º Turno
query = '''
SELECT 
  a.sigla_partido,
  a.id_municipio, 
  sum(a.votos) AS total_votos
FROM 
  `basedosdados.br_tse_eleicoes.resultados_candidato_municipio` a
INNER JOIN `basedosdados.mundo_onu_adh.municipio` b 
ON a.id_municipio = b.id_municipio
WHERE 
  a.ano = 2018 AND cargo = 'presidente' AND turno = 2 AND sigla_uf = 'SP'
GROUP BY 
  sigla_partido, id_municipio
'''
df = bd.read_sql(query, billing_project_id='teste-projeto-bd')

Downloading: 100%|██████████| 1290/1290 [00:00<00:00, 7633.83rows/s]


In [12]:
#Calculando Porcentagem de votação cada municipio 
def calculate_percent(group):
    total_votes = group['total_votos'].sum()
    group['percent'] = group['total_votos'] / total_votes * 100
    return group

df = df.groupby('id_municipio').apply(calculate_percent)

In [13]:
df

,sigla_partido,id_municipio,total_votos,percent
0,PT,3500105,9468,17.298838
1,PSL,3500105,45264,82.701162
2,PT,3500204,1983,24.46336
3,PSL,3500204,6123,75.53664
4,PT,3500303,13005,28.708609
...,...,...,...,...
1285,PSL,3557154,3396,71.239773
1286,PT,3557204,5214,27.635554
1287,PSL,3557204,13653,72.364446
1288,PT,3557303,4185,23.031204
